## Write data for Snowflake workflow

In [1]:
# https://towardsdatascience.com/exploring-bike-share-data-3e3b2f28760c


In [18]:
import urllib
import zipfile
import pandas as pd
import dask.dataframe as dd
import seaborn as sns
import os
import coiled

from dotenv import dotenv_values
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
from dask.distributed import wait, LocalCluster, Client
from dask_snowflake import read_snowflake, to_snowflake
from distributed.client import _get_global_client

In [96]:
connection_kwargs = {
    k.replace("SNOWFLAKE_", "").lower(): v
    for k, v in dotenv_values(".env").items()
}

## Create table if not exists

In [6]:
table_name = "citibike_tripdata"

In [5]:
engine = create_engine(URL(**connection_kwargs))

In [5]:
engine.execute(f"drop table if exists {table_name}");

/var/folders/w6/twsr_k_17m3fbmpgvpnp_v700000gn/T/ipykernel_97362/2050832664.py:1: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  engine.execute(f"drop table if exists {table_name}");


In [7]:
engine.execute(f"""create table if not exists {table_name} (
    ride_id varchar not null unique,
    rideable_type varchar not null,
    started_at timestamp not null,
    ended_at timestamp not null,
    start_station_name varchar not null,
    start_station_id smallint not null,
    end_station_name varchar not null,
    end_station_id smallint not null,
    start_lat number,
    start_lng number,
    end_lat number,
    end_lng number,
    is_member boolean not null
)""")

/var/folders/w6/twsr_k_17m3fbmpgvpnp_v700000gn/T/ipykernel_52849/200109041.py:1: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  engine.execute(f"""create table if not exists {table_name} (


## Create cluster and client

In [20]:
%%time

cluster = coiled.Cluster(
    name="irina-write-to-snowflake",
    n_workers=5,
    backend_options={"region": "us-east-2"}
)

Output()

Output()

CPU times: user 6.27 s, sys: 458 ms, total: 6.73 s
Wall time: 1min 31s


In [7]:
# cluster = LocalCluster()

In [21]:
client = cluster.get_client()

In [22]:
client

<Client: 'tls://10.0.26.250:8786' processes=4 threads=16, memory=59.74 GiB>

## Read the data into dataframe

In [23]:
def safe_int(x):
    try:
        return int(float(x))
    except:
        return -1

In [24]:
start_date = "2019-01-01"
end_date = "2021-12-31"

In [26]:
csv_paths = []
for ts in pd.date_range(start=start_date, end=end_date, freq="MS"):
    csv_paths.append(f"s3://tripdata/{ts.year}{ts.month:02}-*-*.zip")
    csv_paths.append(f"s3://tripdata/{ts.year}{ts.month:02}-*-*.csv.zip")
csv_paths

['s3://tripdata/201901-*-*.zip',
 's3://tripdata/201901-*-*.csv.zip',
 's3://tripdata/201902-*-*.zip',
 's3://tripdata/201902-*-*.csv.zip',
 's3://tripdata/201903-*-*.zip',
 's3://tripdata/201903-*-*.csv.zip',
 's3://tripdata/201904-*-*.zip',
 's3://tripdata/201904-*-*.csv.zip',
 's3://tripdata/201905-*-*.zip',
 's3://tripdata/201905-*-*.csv.zip',
 's3://tripdata/201906-*-*.zip',
 's3://tripdata/201906-*-*.csv.zip',
 's3://tripdata/201907-*-*.zip',
 's3://tripdata/201907-*-*.csv.zip',
 's3://tripdata/201908-*-*.zip',
 's3://tripdata/201908-*-*.csv.zip',
 's3://tripdata/201909-*-*.zip',
 's3://tripdata/201909-*-*.csv.zip',
 's3://tripdata/201910-*-*.zip',
 's3://tripdata/201910-*-*.csv.zip',
 's3://tripdata/201911-*-*.zip',
 's3://tripdata/201911-*-*.csv.zip',
 's3://tripdata/201912-*-*.zip',
 's3://tripdata/201912-*-*.csv.zip',
 's3://tripdata/202001-*-*.zip',
 's3://tripdata/202001-*-*.csv.zip',
 's3://tripdata/202002-*-*.zip',
 's3://tripdata/202002-*-*.csv.zip',
 's3://tripdata/2020

In [81]:
ddf = dd.read_csv(
    csv_paths,
    compression="zip",
    blocksize=None,
    converters={
        "start_station_id": safe_int, 
        "end_station_id": safe_int, 
        "start station id": safe_int, 
        "end station id": safe_int
    }
)
ddf

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
npartitions=72,,,,,,,,,,,,,,,
,int64,object,object,int64,object,float64,float64,int64,object,float64,float64,int64,object,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [31]:
ddf.head(3)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,320,2019-01-01 00:01:47.4010,2019-01-01 00:07:07.5810,3160,Central Park West & W 76 St,40.778968,-73.973747,3283,W 89 St & Columbus Ave,40.788221,-73.970416,15839,Subscriber,1971,1
1,316,2019-01-01 00:04:43.7360,2019-01-01 00:10:00.6080,519,Pershing Square North,40.751873,-73.977706,518,E 39 St & 2 Ave,40.747804,-73.973442,32723,Subscriber,1964,1
2,591,2019-01-01 00:06:03.9970,2019-01-01 00:15:55.4380,3171,Amsterdam Ave & W 82 St,40.785247,-73.976673,3154,E 77 St & 3 Ave,40.773142,-73.958562,27451,Subscriber,1987,1


In [82]:
ddf.dtypes

tripduration                 int64
starttime                   object
stoptime                    object
start station id             int64
start station name          object
start station latitude     float64
start station longitude    float64
end station id               int64
end station name            object
end station latitude       float64
end station longitude      float64
bikeid                       int64
usertype                    object
birth year                   int64
gender                       int64
dtype: object

In [ ]:
"""
ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id        int64
end_station_name       object
end_station_id          int64
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
is_member                bool
"""

In [83]:
ddf = ddf.rename(columns={
    "start station id": "start_station_id",
    "start station name": "start_station_name",
    "start station latitude": "start_lat",
    "start station longitude": "start_lng",
    "end station id": "end_station_id",
    "end station name": "end_station_name",
    "end station latitude": "end_lat",
    "end station longitude": "end_lng",
    "starttime": "started_at",
    "stoptime": "ended_at"
})
ddf.dtypes

tripduration            int64
started_at             object
ended_at               object
start_station_id        int64
start_station_name     object
start_lat             float64
start_lng             float64
end_station_id          int64
end_station_name       object
end_lat               float64
end_lng               float64
bikeid                  int64
usertype               object
birth year              int64
gender                  int64
dtype: object

In [84]:
ddf["ride_id"] = (
    ddf["bikeid"].astype(str) + 
    ddf["usertype"] + 
    ddf["birth year"].astype(str) + 
    ddf["gender"].astype(str) + 
    ddf["tripduration"].astype(str)
)

In [85]:
ddf["is_member"] = ddf["usertype"] == "Subscriber"

In [86]:
ddf = ddf.drop(columns=["bikeid", "usertype", "birth year", "gender", "tripduration"])

In [87]:
ddf.dtypes

started_at             object
ended_at               object
start_station_id        int64
start_station_name     object
start_lat             float64
start_lng             float64
end_station_id          int64
end_station_name       object
end_lat               float64
end_lng               float64
ride_id                object
is_member                bool
dtype: object

## Filter out invalid station ids

In [89]:
ddf2 = ddf[(ddf["start_station_id"] != -1) & (ddf["end_station_id"] != -1)].reset_index(drop=True)

## Create boolean is_member and drop member_casual

In [13]:
#ddf2["is_member"] = ddf2.member_casual == "member"

In [14]:
#ddf2 = ddf2.drop(columns="member_casual")

In [57]:
ddf2 = ddf2.repartition(partition_size="100Mb")

ValueError: The columns in the computed data do not match the columns in the provided metadata
  Extra:   ['ended_at', 'member_casual', 'ride_id', 'rideable_type']
  Missing: ['bikeid', 'birth year', 'gender', 'stoptime', 'tripduration', 'usertype']

In [90]:
ddf2.npartitions

72

In [39]:
ddf2.index.head(3)

RangeIndex(start=0, stop=3, step=1)

In [91]:
ddf2.dtypes

started_at             object
ended_at               object
start_station_id        int64
start_station_name     object
start_lat             float64
start_lng             float64
end_station_id          int64
end_station_name       object
end_lat               float64
end_lng               float64
ride_id                object
is_member                bool
dtype: object

## Store to snowflake

In [92]:
wait(ddf2)

DoneAndNotDoneFutures(done=set(), not_done=set())

In [93]:
to_snowflake(ddf2, name=table_name, connection_kwargs=connection_kwargs)

ValueError: The columns in the computed data do not match the columns in the provided metadata
  Extra:   ['member_casual', 'ride_id', 'rideable_type']
  Missing: ['bikeid', 'birth year', 'gender', 'tripduration', 'usertype']

## Cleanup

In [80]:
client.restart()

<Client: 'tls://10.0.26.250:8786' processes=14 threads=56, memory=209.05 GiB>

In [63]:
cluster.scale(15)

In [94]:
client.close()

In [95]:
cluster.close()